## 止盈止损-TrailingStopLoss
1. 最高价吊灯(highPrice)
2. 波幅吊灯(atrMultiple)
3. 价格百分比跟踪(stairPct)
4. 指标追踪(Indicator)

### 最高价吊灯(highPrice)
适用于日内交易：
1. 设置追踪止损的百分比
2. 创建记录最高最低价的字典并把初始值设置成0与一个极大数值
3. 记录不断创新高或新低的价格
4. 当现价超过最(高/低)价加减追踪止损的百分比的比例即出场

In [ ]:
trailingPct = 0.02 #百分之二

if self.posDict[symbol + "_LONG"] == 0 and self.posDict[symbol + "_SHORT"] == 0:
    self.intraTradeHighDict[symbol] = 0
    self.intraTradeLowDict[symbol] = 999999

# 持有多头仓位
elif self.posDict[symbol + "_LONG"] > 0:
    self.intraTradeHighDict[symbol] = max(self.intraTradeHighDict[symbol], bar.high)
    longStop = self.intraTradeHighDict[symbol] * (1-self.trailingPct)
    if bar.close <= longStop:
        self.sell(symbol, bar.close * 0.985, self.posDict[symbol + "_LONG"])

# 持有空头仓位
elif self.posDict[symbol + "_SHORT"] > 0:
    self.intraTradeLowDict[symbol] = min(self.intraTradeLowDict[symbol], bar.low)
    shortStop = self.intraTradeLowDict[symbol] * (1 + self.trailingPct)
    if bar.close >= shortStop:
        self.cover(symbol, bar.close * 1.015, self.posDict[symbol + "_SHORT"])

### 波幅吊灯(atrMultiple)
适用于日内交易：
1. 设置ATR的倍数，并计算ATR的值。
2. 创建记录最高最低价的字典并把初始值设置成0与一个极大数值
3. 记录不断创新高或新低的价格
4. 当现价超过最(高/低)价加减ATR的倍数即出场，记录止损价，只能提升浮盈。

In [ ]:
atrMultipler = 3 #3倍的ATR

# 计算ATR的值
atr = ta.ATR(am.close, 10)

if self.posDict[symbol + "_LONG"] == 0 and self.posDict[symbol + "_SHORT"] == 0:
    self.intraTradeHighDict[symbol] = 0
    self.intraTradeLowDict[symbol] = 999999
    self.longStop[symbol] = 0
    self.shortStop[symbol] = 999999

# 持有多头仓位
elif self.posDict[symbol + "_LONG"] > 0:
    self.intraTradeHighDict[symbol] = max(self.intraTradeHighDict[symbol], bar.high)
    self.longStop[symbol] = max(self.longStop[symbol], self.intraTradeHighDict[symbol] * (1-self.atrMultipler*self.atr))
    if bar.close <= self.longStop[symbol]:
        self.sell(symbol, bar.close * 0.985, self.posDict[symbol + "_LONG"])

# 持有空头仓位
elif self.posDict[symbol + "_SHORT"] > 0:
    self.intraTradeLowDict[symbol] = min(self.intraTradeLowDict[symbol], bar.low)
    self.shortStop[symbol] = max(self.shortStop[symbol], self.intraTradeLowDict[symbol] * (1+self.atrMultipler*self.atr))
    if bar.close >= self.shortStop[symbol]:
        self.cover(symbol, bar.close * 1.015, self.posDict[symbol + "_SHORT"])

## 价格百分比跟踪(stairPct)
适用于隔夜交易：
1. 设置追踪止损的百分比
2. 创建记录最高最低价的字典并把初始值设置成0与一个极大数值
3. 记录不断创新高或新低的价格
4. 当现价超过最(高/低)价加减追踪止损的百分比的比例即出场

In [ ]:
trailingPct = 0.02 #百分之二

if self.posDict[symbol + "_LONG"] == 0 and self.posDict[symbol + "_SHORT"] == 0:
    self.intraTradeHighDict[symbol] = 0
    self.intraTradeLowDict[symbol] = 999999
    self.n = 0

# 持有多头仓位
elif self.posDict[symbol + "_LONG"] > 0:
    self.intraTradeHighDict[symbol] = max(self.intraTradeHighDict[symbol], bar.high)
    self.n = (self.intraTradeHighDict[symbol]/transactionPrice[symbol]-1)//self.trailingPct
    changePrice = transactionPrice[symbol]*self.n*self.trailingPct
    self.longStop[symbol] = max(self.longStop[symbol], transactionPrice[symbol]*(1-self.trailingPct)+upPrice)
    if bar.close <= self.longStop[symbol]:
        self.sell(symbol, bar.close * 0.985, self.posDict[symbol + "_LONG"])

# 持有空头仓位
elif self.posDict[symbol + "_LONG"] > 0:
    self.intraTradeHighDict[symbol] = max(self.intraTradeHighDict[symbol], bar.high)
    self.n = (self.intraTradeLowDict[symbol]/transactionPrice[symbol]-1)//self.trailingPct
    changePrice = transactionPrice[symbol]*self.n*self.trailingPct
    self.shortStop[symbol] = max(self.shortStop[symbol], transactionPrice[symbol]*(1+self.trailingPct)-changePrice)
    if bar.close <= self.shortStop[symbol]:
        self.cover(symbol, bar.close * 1.015, self.posDict[symbol + "_SHORT"])

### 指标追踪止损(Indicator)
适合信号出场：
1. 设定指标参数
2. 多空出场信号

In [ ]:
kamaPeriod = 20

kama = ta.KAMA(am.close, self.kamaPeriod)
if self.posDict[symbol + "_LONG"] > 0:
    if bar.close<=kama[-1]:
        self.sell(symbol, bar.close * 0.985, self.posDict[symbol + "_LONG"])
if self.posDict[symbol + "_SHORT"] > 0:
    if bar.close>=kama[-1]:
        self.cover(symbol, bar.close * 1.015, self.posDict[symbol + "_SHORT"])

In [ ]:
sarPeriod = 20

sar = ta.SAR(am.close, self.kamaPeriod)
if self.posDict[symbol + "_LONG"] > 0:
    if bar.close<=sar[-1] and bar.close>=sar[-2]:
        self.sell(symbol, bar.close * 0.985, self.posDict[symbol + "_LONG"])
if self.posDict[symbol + "_SHORT"] > 0:
    if bar.close<=sar[-1] and bar.close>=sar[-2]:
        self.cover(symbol, bar.close * 1.015, self.posDict[symbol + "_SHORT"])

### stairPctCase

In [ ]:
"""
固定止损
"""

from __future__ import division
from vnpy.trader.vtConstant import *
from vnpy.trader.app.ctaStrategy.ctaBarManager import CtaTemplate
import numpy as np
import talib as ta
from datetime import timedelta

########################################################################
# 策略继承CtaTemplate
class MultiFrameMaStrategy(CtaTemplate):
    className = 'MultiFrameMaStrategy'
    author = 'ChannelCMT'

    # 策略参数
    barPeriod = 94
    fastWindow = 40     # 快速均线参数
    slowWindow = 70     # 慢速均线参数
    trailingPct = 0.02  #百分之二
    
    # 策略变量
    transactionPrice = {} # 记录成交价格
    maTrend = {}    # 均线趋势，多头1，空头-1
    
    # 参数列表，保存了参数的名称
    paramList = ['className',
                 'author',
                 'barPeriod',
                 'fastWindow',
                 'slowWindow',
                 'stopRatio']  
    
    # 变量列表，保存了变量的名称
    varList = [
               'maTrend',
               'transactionPrice']  
    
    # 同步列表，保存了需要保存到数据库的变量名称
    syncList = ['posDict','eveningDict']

    #----------------------------------------------------------------------
    def __init__(self, ctaEngine, setting):
        # 首先找到策略的父类（就是类CtaTemplate），然后把DoubleMaStrategy的对象转换为类CtaTemplate的对象
        super(MultiFrameMaStrategy, self).__init__(ctaEngine, setting)
    
    #----------------------------------------------------------------------
    def onInit(self):
        """初始化策略（必须由用户继承实现）"""
        self.writeCtaLog(u'双EMA演示策略初始化')
        
        self.setArrayManagerSize(self.barPeriod)
        self.transactionPrice =  {s: 0 for s in self.symbolList}
        self.maTrend = {s: 0 for s in self.symbolList}   # 均线趋势，多头1，空头-1
        self.intraTradeHighDict = {s: 0 for s in self.symbolList}
        self.longStop = {s: 0 for s in self.symbolList}
        self.shortStop = {s: 0 for s in self.symbolList}
        self.intraTradeLowDict = {s: 999999 for s in self.symbolList}
        self.n = {s: 0 for s in self.symbolList}
        
        self.mail("chushihuaaaaaaaaaaaaaaaaaaaaaaaaa")
        self.putEvent()
        
    #----------------------------------------------------------------------
    def onStart(self):
        """启动策略（必须由用户继承实现）"""
        self.writeCtaLog(u'双EMA演示策略启动')
        self.putEvent()
    
    #----------------------------------------------------------------------
    def onStop(self):
        """停止策略（必须由用户继承实现）"""
        self.writeCtaLog(u'双EMA演示策略停止')
        self.putEvent()
        
    #----------------------------------------------------------------------
    def onTick(self, tick):
        """收到行情TICK推送（必须由用户继承实现）"""
        pass
    
    #----------------------------------------------------------------------
    def onBar(self, bar):
        """收到Bar推送（必须由用户继承实现）"""
        symbol = bar.vtSymbol
        # 洗价器
        
        if self.posDict[symbol + "_LONG"] == 0 and self.posDict[symbol + "_SHORT"] == 0:
            self.intraTradeHighDict[symbol] = 0
            self.intraTradeLowDict[symbol] = 999999
            self.n[symbol] = 0

        # 持有多头仓位
        elif self.posDict[symbol + "_LONG"] > 0:
            self.intraTradeHighDict[symbol] = max(self.intraTradeHighDict[symbol], bar.high)
            self.n[symbol] = (self.intraTradeHighDict[symbol]/transactionPrice[symbol]-1)//self.trailingPct
            changePrice = transactionPrice[symbol]*self.n[symbol]*self.trailingPct
            self.longStop[symbol] = max(self.longStop[symbol], transactionPrice[symbol]*(1-self.trailingPct)+changePrice)
            if bar.close <= self.longStop[symbol]:
                self.sell(symbol, bar.close * 0.985, self.posDict[symbol + "_LONG"])

        # 持有空头仓位
        elif self.posDict[symbol + "_LONG"] > 0:
            self.intraTradeHighDict[symbol] = max(self.intraTradeHighDict[symbol], bar.high)
            self.n[symbol] = (self.intraTradeLowDict[symbol]/transactionPrice[symbol]-1)//self.trailingPct
            changePrice = transactionPrice[symbol]*self.n[symbol]*self.trailingPct
            self.shortStop[symbol] = max(self.shortStop[symbol], transactionPrice[symbol]*(1+self.trailingPct)-changePrice)
            if bar.close <= self.shortStop[symbol]:
                self.cover(symbol, bar.close * 1.015, self.posDict[symbol + "_SHORT"])
        print('longStop:', self.longStop[symbol],'shortStop:', self.shortStop[symbol])

    #----------------------------------------------------------------------
    def on60MinBar(self, bar):
        """60分钟K线推送"""
        symbol = bar.vtSymbol
        
        am60 = self.getArrayManager(symbol, "60m")

        if not am60.inited:
            return
        
        # 计算均线并判断趋势
        fastMa = ta.MA(am60.close, self.fastWindow)
        slowMa = ta.MA(am60.close, self.slowWindow)
#         print(fastMa)
        
        if fastMa[-1] > slowMa[-1]:
            self.maTrend[symbol] = 1
        else:
            self.maTrend[symbol] = -1
#         print('maTrend:',self.maTrend)
    
    #----------------------------------------------------------------------
    def on15MinBar(self, bar):
        """收到Bar推送（必须由用户继承实现）"""
        self.cancelAll() # 全部撤单
        symbol = bar.vtSymbol
        
        am15 = self.getArrayManager(symbol, "15m")

        if not am15.inited:
            return

        fastMa = ta.EMA(am15.close, self.fastWindow)
        slowMa = ta.EMA(am15.close, self.slowWindow)

        # 判断买卖
        increaseMa = fastMa[-1]>fastMa[-2] and slowMa[-1]>slowMa[-2]     # 均线上涨
        decreaseMa = fastMa[-1]<fastMa[-2] and slowMa[-1]<slowMa[-2]     # 均线下跌
#         print('increaseMa:',increaseMa)
#         print('decreaseMa:',decreaseMa)         

        
        # 均线上涨和下跌的条件是互斥
        if increaseMa and (self.maTrend[symbol]==1):
            # 如果上涨时手头没有持仓，则直接做多
            if (self.posDict[symbol+'_LONG']==0) and (self.posDict[symbol+'_SHORT']==0):
                self.buy(symbol,bar.close*1.1, 1)
            # 如果有空头持仓，则先平空，再做多
            elif self.posDict[symbol+'_SHORT'] >0:
                self.cancelAll()
                self.cover(symbol,bar.close*1.02, self.posDict[symbol+'_SHORT'])
                self.buy(symbol,bar.close*1.02, 1)

        # 死叉和金叉相反
        elif decreaseMa and (self.maTrend[symbol]==-1):
            if (self.posDict[symbol+'_LONG']==0) and (self.posDict[symbol+'_SHORT']==0):
                self.short(symbol,bar.close*0.98, 1)
            elif self.posDict[symbol+'_LONG'] >0:
                self.cancelAll()
                self.sell(symbol,bar.close*0.98, self.posDict[symbol+'_LONG'])
                self.short(symbol,bar.close*0.98, 1)

#         print(self.posDict)
        # 发出状态更新事件
        self.putEvent()
    
    #----------------------------------------------------------------------
    def onOrder(self, order):
        """收到委托变化推送（必须由用户继承实现）"""
        # 对于无需做细粒度委托控制的策略，可以忽略onOrder
        pass
    
    #----------------------------------------------------------------------
    def onTrade(self, trade):
        """收到成交推送（必须由用户继承实现）"""
        # 对于无需做细粒度委托控制的策略，可以忽略onOrder
        symbol=bar.vtSymbol
        self.transactionPrice[symbol] = trade.price
#         pass
    
    #----------------------------------------------------------------------
    def onStopOrder(self, so):
        """停止单推送"""
        pass